# Homework 3 - What is the best anime in the world?

## 1. Data collection

In [54]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import csv
import re
import nltk
from collections import Counter
import numpy as np

#### List of paths

In [4]:
path_list_anime = "C:/Users/andre/Desktop/list_anime.txt"
url_of_single_page = "https://myanimelist.net/topanime.php?limit="
path_page_general = "C:/Users/andre/Desktop/Anime/Page"
path_documents_file = "C:/Users/andre/Desktop/Anime/documents.tsv"
path_vocabulary_file = "C:/Users/andre/Desktop/Anime/vocabulary.json"
path_index_file = "C:/Users/andre/Desktop/Anime/index.json"

### 1.1. Get the list of animes

#### Write file with all anime urls

In [3]:
# 19130 anime in total, 383 pages with 50 animes

def write_list_anime(path_list_anime, url_of_single_page):
    # Number of total pages
    n_pages = 383 

    with open(path_list_anime, "w", encoding='utf-8') as file:

        for page in tqdm(range(0, n_pages)):

            # URL of the single page 
            url = url_of_single_page + str(page * 50)

            # Get response object
            response = r.get(url)

            # Get html
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all links iterating through the document
            for tag in soup.find_all("tr", class_="ranking-list"):
                a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
                for a in a_list:
                    link = a['href']
                    file.write(str(link) + '\n')

    file.close()

### 1.2. Crawl animes

In [4]:
def download_html(path_list_anime, path_page_general):
    file = open(path_list_anime, "r", encoding='utf-8')

    # Initialize counters 
    counter_anime = 0
    page = 0

    # Get html file for every anime (url is a line in the file list_anime) 
    for line in tqdm(file):
        counter_anime += 1
        if (counter_anime%50 == 1):
            page +=1
        response = r.get(line)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            os.mkdir(path_page_general + str(page))
        except OSError:
            pass

        with open(path_page_general + str(page) + "/article_" + str(counter_anime) + ".html", "w", encoding='utf-8') as file:
            file.write(str(soup))

### 1.3 Parse downloaded pages

#### Function to retrieve all the information from html

In [5]:
# Retrive information at page i of anime j
def retrive_information(i, j):
        with open(path_page_general + str(i) + "/anime" + str(j) + ".html", "r", encoding='utf-8') as file:
    
            # Read file and parse html code
            content_page = file.read()
            soup = BeautifulSoup(content_page, 'html.parser')
            
            # Retrive information 
            animeTitle = soup.find_all("h1", class_="title-name h1_bold_none")[0].find("strong").text
        
            animeType = soup.find_all('a', href = re.compile(r'type*'))[0].text.strip()
            
            animeNumEpisode = soup.find(string='Episodes:').next_element.replace("\n","").strip()
            
            animeNumMembers = soup.find(string='Members:').next_element.replace("\n","").replace(',','').replace('#', '').strip()
        
            animeScore = soup.find('span', {"itemprop":"ratingValue"}).text.strip()
            
            animeUsers = soup.find('span', {"itemprop":"ratingCount"}).text.strip()
            
            animeRank = soup.find(string='Ranked:').next_element.replace("\n","").replace('#','').strip()
            
            animePopularity = soup.find(string='Popularity:').next_element.replace("\n","").replace('#', '').strip()
            
            animeDescription = soup.find('p',{'itemprop':'description'}).text.replace('\n', '').replace('  ', '').strip()

    
            # Iterate in di with dates   
            dates_div = soup.find_all("div", class_="spaceit_pad")
            for i in range(0, len(dates_div)):
        
                # Dates
                if dates_div[i].find("span", string='Aired:') != None:
                    dates = dates_div[i].contents[2].strip().split("to")
                    try:
                        releaseDate = datetime.strptime(dates[0].strip(), '%b %d, %Y').date()
                    except:
                        releaseDate = None
                    try:
                        endDate = datetime.strptime(dates[1].strip(), '%b %d, %Y').date()
                    except:  
                        endDate = None
    
            # Anime Description
            animeDescription = soup.find("p", itemprop='description').text
    
            # Related Animes
            try:
                related_anime_table = soup.find("table", class_="anime_detail_related_anime")
                links_list = [link.text for link in related_anime_table.find_all("a")]
                animeRelated = list(set(links_list))
            except:
                animeRelated = None
    
            # Characters
            animeCharacters = soup.find_all("h3", class_="h3_characters_voice_actors")
            animeCharacters = [character.text for character in animeCharacters]
    
            # Voices
            animeVoices = soup.find_all("td", class_="va-t ar pl4 pr4")
            animeVoices = [actor.find("a").text for actor in animeVoices]
    
            a_no = []
            total = []
            result = []
            animeStaff = []
    
            # Staff 
            if animeCharacters == [] and animeVoices == []:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[0]
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')
    
                smalls = div.find_all("small")
    
                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))
    
                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
                
            # There are no staff
            elif len(soup.find_all("div", class_="detail-characters-list clearfix")) < 2:
                animeStaff = []
                
            # There are one of animeCharacters or animeVoices
            else:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[1]
                
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')

                smalls = div.find_all("small")

                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))

                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
            
        # Create list with overall information
        final_information = [animeTitle, animeType,animeNumEpisode,releaseDate,endDate,
                         animeNumMembers, animeScore,animeUsers,animeRank,animePopularity,
                         animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff]
        return final_information

#### Create a .tsv file for each anime and a .tsv file with 3 most important information for all: Title, Description, Url 

In [6]:
def create_tsvs(path_list_anime, path_page_general, path_documents_file):
    # Create .tsv file
    n_pages = 384

    # Collecting all Urls
    with open(path_list_anime, 'r', encoding='utf-8') as f_urls:
        urls = f_urls.readlines()
    f_urls.close

    # Create .tsv with only 3 important fields
    with open(path_documents_file, 'w', encoding='utf-8') as f:
        tsv_writer = csv.writer(f, delimiter='\t')

        # Write haeders
        tsv_writer.writerow(['animeTitle','animeDescription','Url'])

    f.close

    # Iterating through pages and then animes
    for i in range(1, n_pages):

        # Start and end computed with respect to the 
        start_iterate_anime = (i-1)*50+1
        end_iterate_anime = ((i-1)*50+1)+50
        for j in range(start_iterate_anime, end_iterate_anime):

            # Write for each anime a .tsv file
            with open(path_page_general + str(i) + "/anime_" + str(j)+ ".tsv", 'w', encoding='utf-8') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')

                # Write haeders
                tsv_writer.writerow(['animeTitle', 'animeType','animeNumEpisode','releaseDate','endDate',
                                     'animeNumMembers', 'animeScore','animeUsers','animeRank','animePopularity',
                                     'animeDescription','animeRelated','animeCharacters','animeVoices','animeStaff'])

                # Write information
                information = retrive_information(i, j) 
                tsv_writer.writerow(information)
            out_file.close

            important_information = [information[0],information[10], urls[j]]

            # Append data to .tsv with only 3 important fields
            with open(path_documents_file, 'a', encoding='utf-8') as f:
                tsv_writer = csv.writer(f, delimiter='\t')

                # Write information 
                tsv_writer.writerow(important_information)

            f.close

## 2. Search Engine

### Create pandas dataframe with important information

In [5]:
def df_from_3_information_file(path_documents_file):
    original_df = pd.read_csv(path_documents_file, sep='\t')
    return original_df

### Text cleaning:

In order we will do the following operations:

- Remove "[Written by MAL Rewrite]" at the end of each description

- Remove Contractions (for example "won't" become "will not" and "don't" become "do not")

- Make all characters lower case (for example "Hello" become "hello")

- Remove dashes

- Remove ordinal numbers (for example 1st, 2nd, ...)

- Remove stopwords (adding to stopwords frequent words that appear in descriptions, for example "character", "end") 

- Remove punctuation

- Stemming

In [6]:
import re
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
import string
import json

default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')

# Append some recurrent words in synopsis:
frequent_words = ['character','characters','end']
default_stopwords.extend(frequent_words)

In [7]:
def remove_written_mal_rewrite(text):
    return text.replace('[Written by MAL Rewrite]', '')

In [8]:
def remove_contractions(text):
    # Create patterns
    patterns = [
       (r'won\'t', 'will not'),
       (r'can\'t', 'cannot'),
       (r'i\'m', 'i am'),
       (r'(\w+)\'ll', '\g<1> will'),
       (r'(\w+)n\'t', '\g<1> not'),
       (r'(\w+)\'ve', '\g<1> have'),
       (r'(\w+)\'s', '\g<1> is'),
       (r'(\w+)\'re', '\g<1> are'),
    ]
    
    # Compile patterns with re.compile()
    patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
    # Substitute words
    for (pattern, repl) in patterns:
        text = re.sub(pattern, repl, text)
    return text

In [9]:
def to_lower_case(text):
    return text.lower()

In [10]:
def replace_dashes(text):
    return text.replace('—',' ')

In [11]:
def remove_ordinal_num(text):
    text = re.sub('[\d]+(st|nd|rd|th)', '', text)
    return text

In [12]:
def tokenize_text(text):
    return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

In [13]:
def remove_punctuation(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(text)
    return ' '.join(new_words)

In [14]:
def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

In [15]:
def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

In [16]:
def clean_text(text):
    
    # Text cleaning
    text = remove_written_mal_rewrite(text)
    text = remove_contractions(text)
    text = to_lower_case(text)
    text = replace_dashes(text)
    text = remove_ordinal_num(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    
    return text

In [17]:
def clean_df(original_df):
    df_descriptions = original_df['animeDescription']
    df_descriptions = pd.DataFrame(df_descriptions)
    df_cleaned = df_descriptions.applymap(lambda x : clean_text(x), na_action='ignore')
    return df_cleaned

### 2.1. Conjunctive query

### 2.1.1) Create your index!

#### Create vocabulary and save it in a file .json

In [20]:
def write_vocabulary(path_vocabulary_file):    
    # Create a list 
    df_words = df_descriptions.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    words_union = set()

    for row in df_words['animeDescription']:
        words_union = words_union.union(set(row))

    words_list = list(words_union)

    # Create vocabulary
    vocabulary = { words_list[i] : i for i in range(len(words_list)) }

    # Write the file
    with open(path_vocabulary_file, 'w', encoding='utf-8') as file:
        json.dump(vocabulary, file)
    file.close

#### Create index and save it in a file .json

In [66]:
def write_index(path_index_file):
    # Create the index with (id_word : [doc1, doc2, ...])
    index = {}
    for n_row in range(df_words['animeDescription'].shape[0]):
        for (k,v) in vocabulary.items():
            if v not in index.keys():
                index[v] = []
            if k in df_words['animeDescription'][n_row]:
                # n_row+1 becouse n_anime is n_row_in_dataset + 1
                index[v].append(n_row+1)

    # Write the file
    with open(path_index_file, 'w', encoding='utf-8') as file:
        json.dump(index, file)
    file.close

### 2.1.2) Execute the query

#### Load vocabulary.json and index.json as python dictionaries

In [18]:
def read_vocabulary_and_index(path_vocabulary_file, path_index_file):
    # Vocabulary
    with open(path_vocabulary_file, 'r', encoding='utf-8') as f:
        vocabulary = json.load(f)
    f.close

    # Index
    with open(path_index_file, 'r', encoding='utf-8') as f:
        index = json.load(f)
    f.close
    
    return vocabulary, index

#### Query

In [19]:
def execute_query(vocabulary, index):
    # Query
    query = input()
    query = clean_text(query).split(" ")

    # Trasform every word of the query in the corrispondant id
    id_words_query = [vocabulary[word] for word in query if word in vocabulary.keys()]

    documents_set = set()
    for id_word in id_words_query:
        if documents_set == set():
            documents_set.update(index[str(id_word)])
        else:
            documents_set = documents_set.intersection(set(index[str(id_word)]))
            if documents_set == set():
                break

    documents = list(documents_set)
    documents_in_dataframe = [x-1 for x in documents]

    # Return rows
    original_df = df_from_3_information_file(path_documents_file)
    return original_df.iloc[documents_in_dataframe]

In [34]:
vocabulary, index = read_vocabulary_and_index(path_vocabulary_file, path_index_file)

In [69]:
vocabulary

{'mentor': 0,
 'vast': 1,
 'robot': 2,
 'slice': 3,
 'braun': 4,
 'shouya': 5,
 'sengoku': 6,
 'shouko': 7,
 'function': 8,
 'commot': 9,
 'prove': 10,
 'ignor': 11,
 'shinpachi': 12,
 'mutter': 13,
 'particip': 14,
 'persuad': 15,
 'occult': 16,
 'intrigu': 17,
 'tower': 18,
 'rest': 19,
 'earth': 20,
 'novel': 21,
 'lift': 22,
 'leav': 23,
 'mad': 24,
 'bulli': 25,
 'stellar': 26,
 'occurr': 27,
 'unfortun': 28,
 'mo': 29,
 'neighbor': 30,
 'homag': 31,
 'layer': 32,
 'state': 33,
 'rurouni': 34,
 'comprehens': 35,
 'oper': 36,
 'shot': 37,
 'confound': 38,
 'defin': 39,
 'nonetheless': 40,
 'curs': 41,
 'preserv': 42,
 'grow': 43,
 'shirou': 44,
 'aoba': 45,
 'man': 46,
 'misshapen': 47,
 'emerg': 48,
 'genroku': 49,
 'countri': 50,
 'curtain': 51,
 'highli': 52,
 'tanjir': 53,
 'inflict': 54,
 'telepathi': 55,
 'grown': 56,
 'yoshiwara': 57,
 'adapt': 58,
 'letter': 59,
 'camp': 60,
 'justic': 61,
 'gape': 62,
 'goldenbaum': 63,
 'truli': 64,
 'devic': 65,
 'limit': 66,
 'shogun': 

In [67]:
index

{'0': [24],
 '1': [44],
 '2': [1, 46],
 '3': [14],
 '4': [19],
 '5': [13],
 '6': [31],
 '7': [13],
 '8': [4],
 '9': [5],
 '10': [45],
 '11': [1],
 '12': [2, 6, 15, 16, 25],
 '13': [32],
 '14': [38],
 '15': [36],
 '16': [40],
 '17': [30],
 '18': [17],
 '19': [5, 13, 42],
 '20': [9, 15, 25, 33, 46],
 '21': [4, 14],
 '22': [5],
 '23': [33, 48, 50],
 '24': [4],
 '25': [13, 39, 50],
 '26': [36],
 '27': [40],
 '28': [15, 28, 44],
 '29': [35],
 '30': [1],
 '31': [33],
 '32': [18, 46, 47],
 '33': [34, 36],
 '34': [42],
 '35': [37],
 '36': [28],
 '37': [27],
 '38': [5],
 '39': [12],
 '40': [12],
 '41': [5, 37, 40, 43],
 '42': [36],
 '43': [1, 14, 36, 41],
 '44': [38],
 '45': [26],
 '46': [15, 17, 34, 41, 42, 43],
 '47': [43],
 '48': [2],
 '49': [36],
 '50': [26],
 '51': [36],
 '52': [3, 33],
 '53': [45],
 '54': [20],
 '55': [37],
 '56': [5, 12],
 '57': [9],
 '58': [4],
 '59': [17],
 '60': [26],
 '61': [42],
 '62': [46],
 '63': [8],
 '64': [7],
 '65': [2],
 '66': [27, 36, 39],
 '67': [25],
 '68'

In [43]:
rows = execute_query(vocabulary, index)
rows

 robot


,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n
45,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,https://myanimelist.net/anime/36862/Made_in_Ab...


### 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

Calculate $tfIdf$ as:
$$
\frac{tf_{i,j}}{{|n_{i,j}|}}
$$
where $tf_{i,j}$ is the number of time the word i is present in document j and ${|n_{i,j}|}$ is the lenght of the document (number of total words)

In [26]:
original_df = df_from_3_information_file(path_documents_file)
df_cleaned = clean_df(original_df)

In [31]:
## TF
# cacola term frequency per ogni documento, ma la chiave è la parola, non l'id
tf = []
for num_row in range(len(df_cleaned)):
    doc = df_cleaned.animeDescription.loc[num_row].split()
    len_doc = len(doc)
    counter = Counter(doc)
    for item, count in counter.items():
        counter[item] /= len_doc
    tf.append(counter)
tf

[Counter({'horrif': 0.009009009009009009,
          'alchemi': 0.009009009009009009,
          'experi': 0.009009009009009009,
          'goe': 0.009009009009009009,
          'wrong': 0.009009009009009009,
          'elric': 0.018018018018018018,
          'household': 0.009009009009009009,
          'brother': 0.02702702702702703,
          'edward': 0.036036036036036036,
          'alphons': 0.02702702702702703,
          'left': 0.009009009009009009,
          'catastroph': 0.009009009009009009,
          'new': 0.009009009009009009,
          'realiti': 0.009009009009009009,
          'ignor': 0.009009009009009009,
          'alchem': 0.009009009009009009,
          'principl': 0.009009009009009009,
          'ban': 0.009009009009009009,
          'human': 0.009009009009009009,
          'transmut': 0.009009009009009009,
          'boy': 0.018018018018018018,
          'attempt': 0.009009009009009009,
          'bring': 0.009009009009009009,
          'recent': 0.00900900900900900

In [35]:
## IDF
idf = {}
import numpy as np
numdocs_cont_j = {}
num_docs = len(df_cleaned)
for term_id in index:
    numdocs_cont_j = len(index[term_id])
    idfscore = np.log(num_docs/numdocs_cont_j)
    idf[term_id] = idfscore
idf

{'0': 3.912023005428146,
 '1': 3.912023005428146,
 '2': 3.2188758248682006,
 '3': 3.912023005428146,
 '4': 3.912023005428146,
 '5': 3.912023005428146,
 '6': 3.912023005428146,
 '7': 3.912023005428146,
 '8': 3.912023005428146,
 '9': 3.912023005428146,
 '10': 3.912023005428146,
 '11': 3.912023005428146,
 '12': 2.302585092994046,
 '13': 3.912023005428146,
 '14': 3.912023005428146,
 '15': 3.912023005428146,
 '16': 3.912023005428146,
 '17': 3.912023005428146,
 '18': 3.912023005428146,
 '19': 2.8134107167600364,
 '20': 2.302585092994046,
 '21': 3.2188758248682006,
 '22': 3.912023005428146,
 '23': 2.8134107167600364,
 '24': 3.912023005428146,
 '25': 2.8134107167600364,
 '26': 3.912023005428146,
 '27': 3.912023005428146,
 '28': 2.8134107167600364,
 '29': 3.912023005428146,
 '30': 3.912023005428146,
 '31': 3.912023005428146,
 '32': 2.8134107167600364,
 '33': 3.2188758248682006,
 '34': 3.912023005428146,
 '35': 3.912023005428146,
 '36': 3.912023005428146,
 '37': 3.912023005428146,
 '38': 3.91202

In [36]:
## per mappare le chiavi di tf in numeri(id)
tf_new = []
for d in tf:
    dic = dict((vocabulary[key], value) for (key, value) in d.items())
    tf_new.append(dic)
tf_new

[{1772: 0.009009009009009009,
  1267: 0.009009009009009009,
  481: 0.009009009009009009,
  534: 0.009009009009009009,
  1499: 0.009009009009009009,
  869: 0.018018018018018018,
  1135: 0.009009009009009009,
  667: 0.02702702702702703,
  778: 0.036036036036036036,
  1675: 0.02702702702702703,
  1074: 0.009009009009009009,
  1171: 0.009009009009009009,
  1216: 0.009009009009009009,
  1797: 0.009009009009009009,
  11: 0.009009009009009009,
  1174: 0.009009009009009009,
  100: 0.009009009009009009,
  1531: 0.009009009009009009,
  1874: 0.009009009009009009,
  244: 0.009009009009009009,
  1322: 0.018018018018018018,
  78: 0.009009009009009009,
  1629: 0.009009009009009009,
  333: 0.009009009009009009,
  1123: 0.009009009009009009,
  1026: 0.009009009009009009,
  949: 0.009009009009009009,
  1845: 0.009009009009009009,
  739: 0.009009009009009009,
  926: 0.009009009009009009,
  382: 0.009009009009009009,
  1533: 0.009009009009009009,
  1417: 0.009009009009009009,
  863: 0.018018018018018018,

In [42]:
inverted_index = {}
for term in idf: #for every term_id
    
    for i in range(len(tf_new)): #for every doc
        if int(term) in tf_new[i].keys(): #if term_id is the key of the document_i
            tupla = (i+1, idf[term]*tf_new[i][int(term)]) #note i+1
            
            if term in inverted_index:
                inverted_index[term].append(tupla)
            else:
                inverted_index[term] = []
                inverted_index[term].append(tupla)
inverted_index

{'0': [(24, 0.06630547466827366)],
 '1': [(44, 0.05015414109523264)],
 '2': [(1, 0.028998881305118924), (46, 0.030366753064794343)],
 '3': [(14, 0.0444548068798653)],
 '4': [(19, 0.050805493576988915)],
 '5': [(13, 0.23285851222786583)],
 '6': [(31, 0.04346692228253496)],
 '7': [(13, 0.18628680978229264)],
 '8': [(4, 0.035243450499352666)],
 '9': [(5, 0.0444548068798653)],
 '10': [(45, 0.04602380006386054)],
 '11': [(1, 0.035243450499352666)],
 '12': [(2, 0.060067437208540325),
  (6, 0.09594104554141858),
  (15, 0.02952032170505187),
  (16, 0.02587174261791063),
  (25, 0.024758979494559637)],
 '13': [(32, 0.05216030673904195)],
 '14': [(38, 0.05509891556941051)],
 '15': [(36, 0.0395153838932136)],
 '16': [(40, 0.04770759762717251)],
 '17': [(30, 0.05216030673904195)],
 '18': [(17, 0.06112535945981478)],
 '19': [(5, 0.031970576326818594),
  (13, 0.033492984723333764),
  (42, 0.038539872832329264)],
 '20': [(9, 0.026774245267372626),
  (15, 0.02952032170505187),
  (25, 0.0247589794945596

In [52]:
def docs_contains_query(vocabulary, index):
    subset = execute_query(vocabulary, index)
    indexes = [i+1 for i in subset.index]
    return indexes

In [104]:
def execute_similarity_query(query, k):
    query = "robot mentor"
    query = clean_text(query).split(" ")
    ar = np.zeros(len(vocabulary.keys()))

    ar = np.where(np.equal( vocabulary.keys(), query),1,0)
    print(np.equal(np.array(vocabulary.keys()), query))


[False False]
